In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\Pain_Processed_Files
data = pd.read_csv('PainCateg_Methylome_APPRDXFilter.csv')
data.head(1)

Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\Pain_Processed_Files


,Sentrix,PATNO,HYS,Gender,APPRDX,MDSP_Pain,cg08012045,cg08300930,cg00134295,cg10537824,...,cg12385643,cg14543119,cg11523984,cg04449108,cg14323797,cg02234120,cg23874600,cg27361577,cg00831331,ch.14.849914F
0,200991620021_R01C01,3001,2,1,1,0,0.962437,0.964442,0.24353,0.826172,...,0.79495,0.79222,0.709422,0.906237,0.872806,0.822185,0.925778,0.78207,0.913573,0.045885


In [3]:
X = data.drop(['PATNO','Sentrix','HYS','Gender','MDSP_Pain'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.7391304347826086

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100


BorutaPy finished running.

Iteration: 	35 / 100
Confirmed: 	2
Tentative: 	0
Rejected: 	108272


BorutaPy(estimator=RandomForestClassifier(n_estimators=20,
                                          random_state=RandomState(MT19937) at 0x1B37CA12B40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1B37CA12B40, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  2


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(2)

,CpG,Ranking
0,APPRDX,1
13294,cg12510028,1


In [10]:
Top50.to_csv ('Pain_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('Pain_Boruta_All.csv', index = False)